In [1]:
%matplotlib qt5
import sys
sys.path.append('./pdeSolver')

from domains import *
from equations import *
from torch_solvers import *
import torch

import ipywidgets as widgets
from ipywidgets import interact
from mpl_toolkits import mplot3d
import matplotlib.patches as patches
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d import Axes3D 
import matplotlib.pyplot as plt
import mpltex

cpu


In [3]:
d=3
ps=15
dom_config={"Domain":'FreeSpace',"total_time":1.0}
eqn_config={"N":d,"dim_per_agent":2,"nu":0.01,"lam":1.0}
eqn=HJB_LQR_Unbounded(dom_config,eqn_config)


In [3]:
solver_params={"initial_lr":0.01,
               "net_config":{"net_type":'Normal',
                            "int_layers":[d+ps,d+ps],
                          "batch_norm":True,
                          "y0_net_config":{"int_layers":[d+ps,d+ps],"bias":True},
                          "z0_net_config":{"int_layers":[d+ps,d+ps],"bias":True}},
               "logging_interval":1,
               "dtype":torch.float32,
               "Ndis":120,
               "Nsamp":64,
               "sample_every":1,
               "in_region":True}
sol_normal=Deep_BSDE_Solver(eqn, solver_params)

2.987316717114295


In [4]:
print("Probando Normal Deep BSDE")
solver_params={"initial_lr":0.01,
               "net_config":{"net_type":'Normal',
                            "int_layers":[d+10,d+10],
                          "batch_norm":False,
                          "y0_net_config":{"int_layers":[d+ps,d+ps],"bias":True},
                          "z0_net_config":{"int_layers":[d+ps,d+ps],"bias":True}},
               "logging_interval":100,
               "dtype":torch.float32,
               "Ndis":20,
               "Nsamp":64,
               "sample_every":1,
               "in_region":False}
sol_normal=Deep_BSDE_Solver(eqn, solver_params)
training_normal=sol_normal.train(1000)

Probando Normal Deep BSDE
2.5373364241170857
Epoch  0  y_0  0.26907974  time  0.00931096076965332  error  2.2682566793012593
Epoch  100  y_0  1.2141565  time  0.750197172164917  error  1.323179915671346
Epoch  200  y_0  1.92795  time  1.49159574508667  error  0.6093863995122883
Epoch  300  y_0  2.2829704  time  2.228966236114502  error  0.25436599565028883
Epoch  400  y_0  2.5016706  time  2.9595441818237305  error  0.03566582513332106
Epoch  500  y_0  2.730729  time  3.7139198780059814  error  0.1933926789712932
Epoch  600  y_0  2.9678113  time  4.460423946380615  error  0.43047492193699144
Epoch  700  y_0  3.1975796  time  5.19572377204895  error  0.660243198151591
Epoch  800  y_0  3.4103427  time  5.948390007019043  error  0.8730062692117717
Epoch  900  y_0  3.5914419  time  6.709750413894653  error  1.054105445618632
Epoch  1000  y_0  3.719495  time  7.4539265632629395  error  1.1821586339426067


In [ ]:
sol_normal.save_sol("deep_BSDE_3part.sol")
sol_normal.save_model("deep_BSDE_3part.pth")

In [4]:
print("Probando Merged Deep BSDE")
solver_params={"initial_lr":0.01,
               "net_config":{"net_type":'Merged',
                            "int_layers":[d+20,d+20],
                          "batch_norm":False,
                          "y0_net_config":{"int_layers":[d+ps,d+ps],"bias":True},
                          "z0_net_config":{"int_layers":[d+ps,d+ps],"bias":True}},
               "logging_interval":1,
               "dtype":torch.float32,
               "Ndis":100,
               "Nsamp":100,
               "sample_every":1,
               "in_region":True}
sol_merged=Deep_BSDE_Solver(eqn, solver_params)
training_merged=sol_merged.train(1000)

Probando Merged Deep BSDE
3.0087155944100448
Epoch  0  y_0  0.36329168  time  0.0429377555847168  error  2.645423913597209
Epoch  1  y_0  0.48357046  time  0.08587837219238281  error  2.525145137905223
Epoch  2  y_0  0.61748713  time  0.12839674949645996  error  2.391228461860759
Epoch  3  y_0  0.76942515  time  0.17324018478393555  error  2.239290440677745
Epoch  4  y_0  0.9405769  time  0.2187964916229248  error  2.0681386834374496
Epoch  5  y_0  1.145586  time  0.2658047676086426  error  1.8631295806160995
Epoch  6  y_0  1.4009715  time  0.3070077896118164  error  1.6077440625420638
Epoch  7  y_0  1.7146933  time  0.3483262062072754  error  1.2940222865334579
Epoch  8  y_0  2.0860453  time  0.3892850875854492  error  0.9226703292122909
Epoch  9  y_0  2.5192153  time  0.43302059173583984  error  0.48950024902735434
Epoch  10  y_0  3.0177505  time  0.4777865409851074  error  0.00903490722264566
Epoch  11  y_0  3.5762796  time  0.5225005149841309  error  0.5675640457877091
Epoch  12  y

In [ ]:
sol_merged.save_sol("deep_BSDE_Merged_3part.sol")
sol_merged.save_model("deep_BSDE_Merged_3part.pth")

In [12]:
print("Probando Merged Residual Deep BSDE")
solver_params={"initial_lr":0.01,
               "net_config":{"net_type":'Merged_residual',
                            "int_layers":[15,15],
                          "batch_norm":False,
                          "activation":torch.nn.ELU(),
                          "y0_net_config":{"int_layers":[d+ps,d+ps],"bias":True},
                          "z0_net_config":{"int_layers":[d+ps,d+ps],"bias":True}},
               "logging_interval":100,
               "dtype":torch.float32,
               "Ndis":50,
               "Nsamp":120,
               "sample_every":1,
               "in_region":True}
sol_merged_residual=Deep_BSDE_Solver(eqn, solver_params)
training_merged_residual=sol_merged_residual.train(1000)

Probando Merged Residual Deep BSDE
3.0023485566710604
Epoch  0  y_0  -0.17430027  time  0.02817392349243164  error  3.1766488249634874
Epoch  100  y_0  3.3619795  time  2.9403793811798096  error  0.3596309278870451
Epoch  200  y_0  2.9128978  time  5.815979480743408  error  0.08945073142997151


KeyboardInterrupt: 

In [ ]:
sol_merged_residual.save_sol("deep_BSDE_Merged_Residual_3part.sol")
sol_merged_residual.save_model("deep_BSDE_Merged_Residual_3part.pth")

In [5]:
print("Probando Raissi Deep BSDE")
solver_params={"initial_lr":0.05,
               "net_config":{"int_layers":[15,15,15],
                          "activation":torch.nn.ELU(),
                          "batch_norm":False,
                          "bias":True,
                          "y0_net_config":{"int_layers":[d+ps,d+ps],"bias":True},
                          "z0_net_config":{"int_layers":[d+ps,d+ps],"bias":True}},
               "logging_interval":100,
               "dtype":torch.float32,
               "Ndis":50,
               "Nsamp":100,
               "sample_every":1,
               "in_region":True}
sol_raissi=Raissi_BSDE_Solver(eqn, solver_params)
#sol_raissi.true_sol=-2.4
training_raissi=sol_raissi.train(1000)

Probando Raissi Deep BSDE
2.999075482159979
Epoch  0  y_0  -0.11830184  time  0.03306460380554199  error  3.1173773207963773
Epoch  100  y_0  3.6294065  time  3.5408377647399902  error  0.6303309700189761
Epoch  200  y_0  2.2313287  time  7.006427049636841  error  0.7677467563451597
Epoch  300  y_0  0.76748264  time  10.489849090576172  error  2.231592843800909
Epoch  400  y_0  -0.1664215  time  13.954040765762329  error  3.165496984988577
Epoch  500  y_0  -0.25928634  time  17.49099564552307  error  3.25836182621134
Epoch  600  y_0  -0.39208326  time  20.961024522781372  error  3.3911587395967313
Epoch  700  y_0  -0.51859564  time  24.430195331573486  error  3.5176711180509397
Epoch  800  y_0  -0.3978628  time  27.891407251358032  error  3.3969382741750547
Epoch  900  y_0  -0.35543317  time  31.354418516159058  error  3.354508648187048
Epoch  1000  y_0  -0.43059447  time  34.82421660423279  error  3.4296699562372037


In [ ]:
sol_raissi.save_sol("Raissi_3part.sol")
sol_raissi.save_model("Raissi_3part.pth")

In [7]:
sol=load_sol("deep_BSDE_3part.sol")
sol.load_model("deep_BSDE_3part.pth")
#sol.train(1000)

Calcula
inf


In [18]:
sol.training_history

[[0, 0.012281254, -2.4642875567492997, 0.19138026237487793],
 [10, 2.1636982, -0.3128706142720734, 1.383885383605957],
 [20, 4.12646, 1.6498912646952117, 2.5616252422332764],
 [30, 2.8058133, 0.32924450184731135, 3.758310079574585],
 [40, 3.4700375, 0.9934686496439422, 4.959244012832642],
 [50, 3.3034542, 0.8268853500071014, 6.176784992218018],
 [60, 3.34761, 0.8710411861124481, 7.352210760116577],
 [70, 3.3718812, 0.8953124358835662, 8.524930715560913],
 [80, 3.197638, 0.7210692241373504, 9.707330703735352],
 [90, 3.378092, 0.9015232398691619, 10.89603567123413],
 [100, 3.2569668, 0.7803980186167205, 12.076613426208496],
 [110, 3.2122626, 0.7356938197794403, 13.25002670288086],
 [120, 3.272395, 0.7958260848703826, 14.45573902130127],
 [130, 3.0476131, 0.5710443332376922, 15.675055027008057],
 [140, 3.3898776, 0.9133087470713104, 16.94320273399353],
 [150, 3.0708272, 0.594258435029074, 18.140135526657104],
 [160, 3.2169378, 0.7403689696970428, 19.314545392990112],
 [170, 3.1417072, 0.6

In [ ]:
training_normal.shape

(1001, 4)

In [ ]:
def moving_average(a, n=3) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

In [ ]:

trainings=[training_normal]
names=['Deep BSDE']


def plot_training(trainings,p):
    fig, ax = plt.subplots()
    for i in range(len(trainings)):
        err=moving_average(trainings[i][:,2],n=10)
        ax.plot(trainings[i][:err.size,p],err,label=names[i])
    if p==0:
        ax.set_xlabel("Epochs")
    else:
        ax.set_xlabel("Time")
    ax.set_ylabel("error")
    ax.legend()
    #fig.savefig('training')

    #plt.show()

plot_training(trainings,0)    

In [ ]:
sol.simulate_trajectory(0.0,torch.tensor([0.1,0.2,0.7,0.2,0.1,0.9]))

array([[0.1       , 0.2       , 0.69999999, 0.2       , 0.1       ,
        0.89999998],
       [0.09537392, 0.20231915, 0.69141741, 0.18422391, 0.11775233,
        0.88852527],
       [0.07391227, 0.20773814, 0.67157512, 0.1959082 , 0.11898015,
        0.89309429],
       [0.07491391, 0.2047241 , 0.65612323, 0.21634636, 0.1329384 ,
        0.87890105],
       [0.06482076, 0.21199511, 0.67183698, 0.21879616, 0.1394546 ,
        0.84982297],
       [0.08353942, 0.21965516, 0.66894809, 0.22257091, 0.15769689,
        0.86128369],
       [0.06782776, 0.21758375, 0.68214305, 0.22964555, 0.17882364,
        0.86329099],
       [0.08306491, 0.23635313, 0.6711111 , 0.22404943, 0.20496856,
        0.87794078],
       [0.10530361, 0.2155944 , 0.68540859, 0.22471307, 0.20217828,
        0.90354094],
       [0.11285543, 0.21697848, 0.68719572, 0.24550331, 0.21107952,
        0.92315102],
       [0.09345163, 0.20580125, 0.68151582, 0.24397857, 0.1936126 ,
        0.91418859],
       [0.09726784, 0

In [ ]:
sol.plot_solution()

ValueError: not enough values to unpack (expected 2, got 1)

In [3]:
print("Probando Raissi BSDE")
d=100
dom_config={"Domain":'FreeSpace',"total_time":1.0}
eqn_config={"N":d,"dim_per_agent":1,"nu":1.0,"lam":1.0}
eqn=HJB_LQR_Unbounded(dom_config,eqn_config)

solver_params={"initial_lr":0.01,
               "net_config":{"int_layers":[2*d,2*d,2*d],
                          "activation":torch.nn.ELU(),
                          "bias":True,
                          "batch_norm":False},
               "logging_interval":200,
               "dtype":torch.float32,
               "Ndis":20,
               "Nsamp":64,
               "sample_every":1,
               "in_region":True}

sol=Raissi_BSDE_Solver(eqn, solver_params)

Probando Raissi BSDE


KeyboardInterrupt: 

In [ ]:
sol.train(1000)

Epoch  1200  y_0  4.489172  time  8.050663948059082  loss  901.9514  error  -0.09184456
Epoch  1400  y_0  4.4985986  time  15.922204971313477  loss  779.19836  error  -0.082417965
Epoch  1600  y_0  4.498658  time  23.30868434906006  loss  682.9683  error  -0.08235836
Epoch  1800  y_0  4.507114  time  30.64812421798706  loss  608.0607  error  -0.07390261
Epoch  2000  y_0  4.5049334  time  38.412628412246704  loss  549.11426  error  -0.07608318


array([[ 0.00000000e+00,  1.71332734e+05, -1.69154592e-02,
        -4.59793186e+00,  6.78303242e-02],
       [ 2.00000000e+02,  5.06042109e+04,  2.31701517e+00,
        -2.26400137e+00,  8.12673783e+00],
       [ 4.00000000e+02,  1.42419883e+04,  3.49748850e+00,
        -1.08352804e+00,  1.57756758e+01],
       [ 6.00000000e+02,  3.68830005e+03,  4.12826347e+00,
        -4.52753067e-01,  2.38735039e+01],
       [ 8.00000000e+02,  1.43117639e+03,  4.38441849e+00,
        -1.96598053e-01,  3.12879910e+01],
       [ 1.00000000e+03,  9.54505554e+02,  4.47110558e+00,
        -1.09910965e-01,  3.86455989e+01],
       [ 1.99000000e+02,  9.01951416e+02,  4.48917198e+00,
        -9.18445587e-02,  8.05066395e+00],
       [ 3.99000000e+02,  7.79198364e+02,  4.49859858e+00,
        -8.24179649e-02,  1.59222050e+01],
       [ 5.99000000e+02,  6.82968323e+02,  4.49865818e+00,
        -8.23583603e-02,  2.33086843e+01],
       [ 7.99000000e+02,  6.08060730e+02,  4.50711393e+00,
        -7.39026070e-02

In [ ]:
print("Probando interp")
dom_config={"total_time":1.0,"pInf":0.4,"pSup":0.6}
dom=EmptyRoom(dom_config)
eqn_config={"N":1,"nu":0.05,"lam":4.0}
eqn=HJB_LQR_Equation_2D(dom,eqn_config)

solver_params={"initial_lr":0.001,
            "initial_loss_weigths":[2.0,1.0,1.0,1.0],
            "net_size":{"width":30,"depth":3,"weigth_norm":False},
            "logging_interval":10,
            "dtype":torch.float32,
            "dt":0.01,
            "N_max":25,
            "N_samples_per_batch_interior":200,
            "N_samples_per_batch_boundary":40,
            "sample_every":1,
            "alpha":0.09,
            "adaptive_weigths":False}
sol=Interp_PINN_BSDE_solver(eqn,solver_params)



Probando interp


In [ ]:
sol.plot_N_agent_sample_path()

In [ ]:
sol.train(600)

Comencemos a calcular
1
Li: 0.09841418266296387
Lb: 0.0006737709045410156
Tiempto toal 0.26204895973205566
2
Li: 0.09267091751098633
Lb: 0.0005962848663330078
Tiempto toal 0.2795860767364502
3
Li: 0.0906825065612793
Lb: 0.00054931640625
Tiempto toal 0.277707576751709
4
Li: 0.09751653671264648
Lb: 0.0005617141723632812
Tiempto toal 0.33020782470703125
5
Li: 0.1075289249420166
Lb: 0.0015707015991210938
Tiempto toal 0.31074023246765137
6
Li: 0.10314512252807617
Lb: 0.0006349086761474609
Tiempto toal 0.2967071533203125
7
Li: 0.09302091598510742
Lb: 0.00054168701171875
Tiempto toal 0.28818511962890625
8
Li: 0.09466767311096191
Lb: 0.0005559921264648438
Tiempto toal 0.2933237552642822
9
Li: 0.0931704044342041
Lb: 0.0005035400390625
Tiempto toal 0.2884104251861572
10
Li: 0.10255646705627441
Lb: 0.0005385875701904297
Tiempto toal 0.2972695827484131
Li: 0.09607243537902832
Lb: 0.003363370895385742
Epoch  10  time  3.025930404663086  loss  [9.722609]  L1: 4.821680068969727  L2: 0.030339641496539

/home/carlos/Documentos/Trabajo de grado/Tesis/code/./pdeSolver/torch_solvers.py:457: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(training_history)


array([[10, array([9.722609], dtype=float32), 3.025930404663086],
       [20, array([9.68509], dtype=float32), 6.125405311584473],
       [30, array([9.272011], dtype=float32), 9.334819078445435],
       [40, array([9.5821705], dtype=float32), 12.530322074890137],
       [50, array([9.202223], dtype=float32), 15.928580045700073],
       [60, array([9.462687], dtype=float32), 19.221311569213867],
       [70, array([9.3215885], dtype=float32), 22.603657007217407],
       [80, array([8.781427], dtype=float32), 26.0795636177063],
       [90, array([9.0949545], dtype=float32), 29.615532159805298],
       [100, array([8.708914], dtype=float32), 33.090713024139404],
       [110, array([8.432859], dtype=float32), 36.37538266181946],
       [120, array([8.900941], dtype=float32), 39.79157519340515],
       [130, array([8.670815], dtype=float32), 43.23753809928894],
       [140, array([8.535451], dtype=float32), 46.56500864028931],
       [150, array([8.37327], dtype=float32), 49.92069911956787]

In [ ]:
sol=load_sol("/home/carlos/Documentos/Trabajo de grado/Tesis/code/LQR_INTERP_N_=1_solution.sol")
sol.load_model("/home/carlos/Documentos/Trabajo de grado/Tesis/code/models/LQR_INTERP_N=1_model.pth")
#dom.plot_N_brownian_paths(np.sqrt(0.05),0.01,2000,0.0,[0.1,0.1],1,False,False)

In [ ]:
sol.plot_solution()